In [2]:
from _02_making_test_set import start_train_set, start_test_set

housing = start_train_set.drop("median_house_value", axis=1)
housing_labels = start_train_set["median_house_value"].copy

본격적으로 사용할 feature와 label 분리

In [3]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

싸이킷런이 제공하는 **SimpleImputer** 클래스를 사용하여 nan값을 대체하는 법 설명

In [4]:
housing_num = housing.drop("ocean_proximity", axis=1)

imputer가 숫자형 데이터만 다룰 수 있기 때문에 카테고리값인 특성은 잠시 제외

In [5]:
imputer.fit(housing_num)

SimpleImputer(strategy='median')

In [6]:
imputer.statistics_

array([-118.49  ,   34.26  ,   29.    , 2123.5   ,  434.    , 1166.    ,
        409.    ,    3.5341])

In [7]:
housing_num.median().values

array([-118.49  ,   34.26  ,   29.    , 2123.5   ,  434.    , 1166.    ,
        409.    ,    3.5341])

In [8]:
X = imputer.transform(housing_num)

각 특성의 median값으로 설정된 imputer를 이용해 각 특성의 nan값을 대체

In [9]:
import pandas as pd
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

치환된 데이터셋을 다시 pandas 데이터프레임으로 변환

In [10]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

,ocean_proximity
5288,<1H OCEAN
12865,INLAND
9174,<1H OCEAN
17247,<1H OCEAN
14138,NEAR OCEAN
5722,<1H OCEAN
18559,NEAR OCEAN
18488,INLAND
9796,NEAR OCEAN
19705,INLAND


해당 데이터셋에서 카테고리 데이터는 'ocean_proximity' 1개 뿐.  
그래서 column이름을 적을 때, 기존 차원을 유지하기 위해서 **리스트에 담아서 가져오는 것** 주목.  

In [11]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[0.],
       [1.],
       [0.],
       [0.],
       [4.],
       [0.],
       [4.],
       [1.],
       [4.],
       [1.]])

범주형 데이터를 숫자에 매핑하여 변환해주는 **OrdinalEncoder** 클래스 사용  
해당 클래스의 **fit_transform** 메서드를 이용하여 해당 데이터에 대해 fit과 transform을 동시에 진행

In [12]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

이 방법은 가까이 있는 두 값을 떨어져 있는 두 값보다 더 비슷하다고 학습하게 됨.   
만약 카테고리가 'bad', 'average', 'good', 'excellent' 와 같이 순서가 있다면 괜찮음.  
하지만 'ocean_proximity'는 그것에 해당하지 않음. 이때는 **one-hot encoding** 방식이 더 좋음

In [13]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [14]:
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.]])

In [16]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

---  

사이킷런이 제공하는 기본 베이스를 활용하여 자신만의 변환기를 만들 수도 있음  
**fit(), transform(), fit_transform()** 메서드를 구현한 파이썬 클래스를 만들면 됨  
**fit_transform** 메서드는 **TransformerMixin**을 상속하면 자동으로 생성됨  
또한 **BaseEstimator**를 상속하면 하이퍼파라미터 튜닝에 필요한 **get_params()와 set_params()** 메서드를 추가로 얻음

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

rooms_ix, bedrooms_ix, pop_ix, house_ix = 3, 4, 5, 6

class CombinedAttribsAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_house = X[:, rooms_ix] / X[:, house_ix]
        pop_per_house = X[:, pop_ix] / X[:, house_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_house, pop_per_house, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_house, pop_per_house]
attr_adder = CombinedAttribsAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [27]:
np.c_[np.array([[0, 0, 1], [1, 0, 0]]), np.array([[2, 2, 2], [3, 3, 3]])]

array([[0, 0, 1, 2, 2, 2],
       [1, 0, 0, 3, 3, 3]])

In [28]:
np.c_[np.array([[1,2,3]]), 0, 0, np.array([[4,5,6]])]

array([[1, 2, 3, 0, 0, 4, 5, 6]])

시간을 절약할 수 있도록 다향한 기능을 가진 변환기를 만들어 놓으면 좋음  
**np.c_**는 concatenate 기능이라고 보면 됨

이 밖에 데이터에 적용할 가장 중요한 변환 중 하나가 **특성 스케일링**임  
거의 모든 머신러닝 알고리즘은 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않음  
대표적인 간단한 스케일링 방법 두 가지가 있음  
> **min_max 스케일링**(정규화)  
> **표준화**  
  
**min_max 스케일링은 이상치에 영향을 많이 받음  
표준화는 이상치에 영향을 덜 받지만 범위의 상한과 하한이 없어서 어떤 알고리즘에 문제가 될 수도 있음**  
  
> 또한 모든 변환기에서 스케일링은 전체 데이터에 대해서가 아닌 훈련 데이터에 대해서만 적용해야 함  
> 훈련 데이터에 대해 fit한 변환기로 테스트 데이터를 transform해야 함  
  
아래 예시는 표준화를 위한 사이킷런 클래스를 활용함

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttribsAdder()),
    ("std_scaler", StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

연속된 변환기를 순서대로 연결해주는 **Pipeline**클래스가 있음  
이름/추정기 쌍의 목록을 입력으로 받음. 이름은 뭐가 돼도 상관없음  

> **가장 마지막 단계에는 변환기와 추정기 모두 사용 가능**  
> 그 외의 단계에는 **변환기만** 사용가능: 즉 fit_transform 메서드를 갖고 있어야 함  
  
Pipeline의 fit()메서드를 호출하면 모든 변환기의 fit_transform()메서드를 순서대로 호출하면서 다음 단계의 입력으로 전달함  


In [30]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num) # names of num_columns
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("tf_num", num_pipeline, num_attribs),
    ("tf_cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

pandas 데이터프레임의 각 columns마다 다른 변환을 적용하고 싶을 때를 위해 **ColumnTransformer** 클래스 사용  
> **(변환기이름/ 변환기/ 적용할 columns이름)** 쌍을 입력 받음
  
기본적으로 나열되지 않은 column은 삭제됨;;  
  
위 예시에서 OneHotEncoder클래스는 희소행렬을 반환하고 num_pipeline은 밀집행렬을 반환함  
이 경우 ColumnsTransformer는 최종 행렬의 밀집 정도를 추정하여 밀집도가 0.3 이하면 희소행렬을 반환하고 그 이상이면 밀집 행렬을 반환함  
지금은 밀집 행렬을 반환한다고 함  

최종 출력된 데이터의 타입은 **np.array**임